In [1]:
import PIL
import torch
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from Dataset.AerialDataset import AerialDataset


- Generar imagenes bicubicas
- Construir Dataset
- Construir Dataloader
- SR3
- SRdiff
- SR3+

## 64 -> 256


#### Generación de imagenes bicubicas

In [ ]:
import os
import numpy as np
from scipy.interpolate import interpn
from PIL import Image
from tqdm import tqdm

##TODO Mover esto a su propio modulo donde poder usarlo para preparar los datos a posteriori.
## TODO cambiar metodo para q vaya más rapido.


def bicubic_interpolation(image, objective_dim): # De momento lo implementare para 1 sola foto a la vez
    #Calculo nuevas dimensiones
    height, width = image.shape[0] , image.shape[1]
    new_width, new_height = objective_dim[0], objective_dim[1]
    new_image = np.zeros((new_height, new_width, image.shape[2]))
    
     # Generar cuadrículas para las coordenadas X e Y de la imagen original y la interpolada
    x = np.linspace(0, width - 1, width)
    y = np.linspace(0, height - 1, height)
    new_x = np.linspace(0, width - 1, new_width)
    new_y = np.linspace(0, height - 1, new_height)
    new_image = interpn((y, x), image, (new_y[:,None], new_x), method='cubic', bounds_error=False, fill_value=0)
    return new_image

dataset_dir = 'E:\\TFG\\air_dataset\\64'
for image_file in tqdm(os.listdir(dataset_dir)):
    image = Image.open(os.path.join(dataset_dir, image_file))
    image = np.array(image)
    interpolated = bicubic_interpolation(image, (256, 256))
    interpolated_image = Image.fromarray(interpolated.astype(np.uint8))
    interpolated_image.save(f'E:\\TFG\\air_dataset\\sr\\64_256\\{image_file}')
    

## Entrenamiento

In [2]:
lr_size = 64
hr_size = 256
batch_size = 64
epochs = 5
learning_rate = 0.001
dataset_dir = 'E:\\TFG\\air_dataset'

logs_dir = 'E:\\TFG\\logs'

In [3]:
dataset = AerialDataset(dataset_dir, lr_size, hr_size)
train_dataset, val_dataset, test_dataset = random_split(dataset, [0.6, 0.2, 0.2])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cuda


### SRDiff

#### Modelo

In [11]:
from models.SRDiff.diffusion import GaussianDiffusion
from models.SRDiff.diffsr_modules import Unet, RRDBNet

hidden_size = 64
dim_mults = [1,2,2,4]
rrdb_num_features = 32
rrdb_num_blocks = 8
timesteps = 100
losstype = 'l1'

denoise_fn = Unet(
    hidden_size, out_dim=3, cond_dim=rrdb_num_features, dim_mults=dim_mults, rrdb_num_block=rrdb_num_blocks)

rrdb = RRDBNet(3, 3, rrdb_num_features, rrdb_num_blocks, rrdb_num_features// 2)

model = GaussianDiffusion(
    denoise_fn=denoise_fn,
    rrdb_net=rrdb,
    timesteps= timesteps,
    loss_type=losstype
)

model.to(device)

GaussianDiffusion(
  (denoise_fn): Unet(
    (cond_proj): ConvTranspose2d(96, 64, kernel_size=(16, 16), stride=(8, 8), padding=(4, 4))
    (time_pos_emb): SinusoidalPosEmb()
    (mlp): Sequential(
      (0): Linear(in_features=64, out_features=256, bias=True)
      (1): Mish()
      (2): Linear(in_features=256, out_features=64, bias=True)
    )
    (downs): ModuleList(
      (0): ModuleList(
        (0): ResnetBlock(
          (mlp): Sequential(
            (0): Mish()
            (1): Linear(in_features=64, out_features=64, bias=True)
          )
          (block1): Block(
            (block): Sequential(
              (0): ReflectionPad2d((1, 1, 1, 1))
              (1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
              (2): Mish()
            )
          )
          (block2): Block(
            (block): Sequential(
              (0): ReflectionPad2d((1, 1, 1, 1))
              (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
              (2): Mish()
            )

Define varios parámetros para la construcción del modelo:

- hidden_size: Tamaño oculto para la red Unet.
- dim_mults: Lista de multiplicadores de dimensiones para la red Unet.
- rrdb_num_features: Número de características para los bloques RRDB.
- rrdb_num_blocks: Número de bloques RRDB en la red RRDBNet.
- timesteps: Número de pasos de tiempo para el proceso de difusión gaussiana.
- losstype: Tipo de pérdida a utilizar (en este caso, 'l1' para la pérdida L1).

Crea una instancia de la red Unet para el proceso de denoising. La red Unet toma como entrada una imagen ruidosa y produce una imagen denoised. Se especifican los parámetros como el tamaño oculto, la dimensión de salida, la dimensión condicional y los multiplicadores de dimensión.

Crea una instancia de la red RRDBNet para el proceso de super-resolución. La red RRDBNet toma una imagen de baja resolución como entrada y produce una imagen de alta resolución como salida. Se especifican los parámetros como el número de canales de entrada y salida, el número de características por bloque RRDB, el número de bloques RRDB, los multiplicadores de dimensión y el número de canales por grupo en las capas de bloque RRDB.

Crea una instancia del modelo de difusión gaussiana, que integra la red de denoising (Unet) y la red de super-resolución (RRDBNet). Se especifican los parámetros como la función de denoising, la red de super-resolución, el número de pasos de tiempo para la difusión gaussiana y el tipo de pérdida a utilizar. Esta instancia del modelo se utilizará para entrenar y evaluar el modelo completo en tareas de super-resolución de imágenes mediante difusión gaussiana.

#### Optimizador y scheduler

In [12]:
lr= 0.0002
decay_steps= 100000
gamma = 0.5

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=decay_steps, gamma=gamma)

In [13]:
from tasks.Trainer import Trainer


ModuleNotFoundError: No module named 'tensorboard'